<a href="https://colab.research.google.com/github/maxmatical/fast.ai/blob/master/fastai_hf_self_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install fastai==2.2.7 # as of may 1 2020 fastai 2.3.0 breaks blurr
!pip install transformers
!pip install ohmeow-blurr==0.0.22

In [2]:
from fastai.test_utils import show_install
show_install()



```text
=== Software === 
python        : 3.7.10
fastai        : 2.2.7
fastcore      : 1.3.19
fastprogress  : 0.2.7
torch         : 1.7.1
nvidia driver : 460.32
torch cuda    : 10.2 / is available
torch cudnn   : 7605 / is enabled

=== Hardware === 
nvidia gpus   : 1
torch devices : 1
  - gpu0      : Tesla T4

=== Environment === 
platform      : Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
distro        : #1 SMP Thu Jul 23 08:00:38 PDT 2020
conda env     : Unknown
python        : /usr/bin/python3
sys.path      : 
/content
/env/python
/usr/lib/python37.zip
/usr/lib/python3.7
/usr/lib/python3.7/lib-dynload
/usr/local/lib/python3.7/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.7/dist-packages/IPython/extensions
/root/.ipython
```

Please make sure to include opening/closing ``` when you paste into forums/github to make the reports appear formatted as code sections.

Optional package(s) to enhance the diagnostics can be installed with:
pip install distro
Once in

In [3]:
from transformers import *
from fastai.text.all import *
from blurr.data.all import *
from blurr.modeling.all import *
import torch.nn as nn



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
path = untar_data(URLs.IMDB_SAMPLE)

model_path = Path('models')
imdb_df = pd.read_csv(path/'texts.csv')

imdb_df.shape

(1000, 3)

In [5]:
imdb_df.head(5)

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False


In [6]:
pretrained_model_name = 'distilroberta-base'
hf_model_cls = RobertaForSequenceClassification
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name,
                                                                               model_cls=hf_model_cls)

In [7]:
blocks = (HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), CategoryBlock)
dblock = DataBlock(blocks=blocks,  get_x=ColReader('text'), get_y=ColReader('label'), splitter=ColSplitter())

dls = dblock.dataloaders(imdb_df, bs=4)

In [8]:
dls.show_batch(dataloaders=dls, max_n=2)

,text,category
0,"Raising Victor Vargas: A Review<br /><br />You know, Raising Victor Vargas is like sticking your hands into a big, steaming bowl of oatmeal. It's warm and gooey, but you're not sure if it feels right. Try as I might, no matter how warm and gooey Raising Victor Vargas became I was always aware that something didn't quite feel right. Victor Vargas suffers from a certain overconfidence on the director's part. Apparently, the director thought that the ethnic backdrop of a Latino family on the lower east side, and an idyllic storyline would make the film critic proof. He was right, but it didn't fool me. Raising Victor Vargas is the story about a seventeen-year old boy called, you guessed it, Victor Vargas (Victor Rasuk) who lives his teenage years chasing more skirt than the Rolling Stones could do in all the years they've toured. The movie starts off in `Ugly Fat' Donna's bedroom where Victor is sure to seduce her, but a cry from outside disrupts his plans when his best-friend Harold (Kevin Rivera) comes-a-looking for him. Caught in the attempt by Harold and his sister, Victor Vargas runs off for damage control. Yet even with the embarrassing implication that he's been boffing the homeliest girl in the neighborhood, nothing dissuades young Victor from going off on the hunt for more fresh meat. On a hot, New York City day they make way to the local public swimming pool where Victor's eyes catch a glimpse of the lovely young nymph Judy (Judy Marte), who's not just pretty, but a strong and independent too. The relationship that develops between Victor and Judy becomes the focus of the film. The story also focuses on Victor's family that is comprised of his grandmother or abuelita (Altagracia Guzman), his brother Nino (also played by real life brother to Victor, Silvestre Rasuk) and his sister Vicky (Krystal Rodriguez). The action follows Victor between scenes with Judy and scenes with his family. Victor tries to cope with being an oversexed pimp-daddy, his feelings for Judy and his grandmother's conservative Catholic upbringing.<br /><br />The problems that arise from Raising Victor Vargas are a few, but glaring errors. Throughout the film you get to know certain characters like Vicky, Nino, Grandma,",negative
1,"Well, what can I say.<br /><br />""What the Bleep do we Know"" has achieved the nearly impossible - leaving behind such masterpieces of the genre as ""The Postman"", ""The Dungeon Master"", ""Merlin"", and so fourth, it will go down in history as the single worst movie I have ever seen in its entirety. And that, ladies and gentlemen, is impressive indeed, for I have seen many a bad movie.<br /><br />This masterpiece of modern cinema consists of two interwoven parts, alternating between a silly and contrived plot about an extremely annoying photographer, abandoned by her husband and forced to take anti-depressants to survive, and a bunch of talking heads going on about how quantum physics supposedly justifies their new-agy pseudo-philosophy. Basically, if you start your day off meditating to the likes of Enya and Kenny G, this movie is for you. If you have a sense of humor, a crowd of people who know how to have fun, and a sizable portion of good weed, then this movie is for you as well. Otherwise, stay away. Take my word for it.<br /><br />The first thing that struck me about ""What the Bleep do you Know"" is that is seemed to be edited and put together by the same kinds of people that shoot cheap weddings on camera, complete with pink heart effects, computer-generated sparkles across the screen, and other assorted silliness. Who let these people anywhere near a theatrical release is a mystery to me. I guess this is what too much Kenny G does to you. The movie was permeated with cheesy GCI, the likes that you or I can produce on our own computer via over-the-counter video editing software, but never would, because it's just way too ridiculous.<br /><br />The script was _obviously_ written by someone with n

### Training 1st model

In [9]:
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                loss_func=LabelSmoothingCrossEntropyFlat(),
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter).to_fp16()

learn.unfreeze()

cbs = [
    GradientAccumulation(n_acc=32),
    ReduceLROnPlateau(monitor='accuracy', factor=5, patience=5),
    EarlyStoppingCallback(monitor='accuracy', min_delta=0.01, patience=7),
    SaveModelCallback(monitor="accuracy", fname="model_1")
]



In [10]:
learn.fit_one_cycle(
    3,
    lr_max=3e-5,
    cbs = cbs
)



epoch,train_loss,valid_loss,accuracy,time
0,0.684760,0.644294,0.555000,00:15
1,0.408468,0.405541,0.900000,00:15
2,0.334760,0.393769,0.900000,00:15


Better model found at epoch 0 with accuracy value: 0.5550000071525574.
Better model found at epoch 1 with accuracy value: 0.8999999761581421.


### generate psuedo-labels for both train and valid data

In [11]:
test_dl = learn.dls.test_dl(list(imdb_df["text"]))

In [18]:
pred_proba, _ = learn.get_preds(dl=test_dl, with_decoded=False)

In [19]:
pred_proba = pred_proba.cpu().numpy()

In [20]:
pred_proba.shape

(1000, 2)

In [24]:
pred_proba[1]

array([0.253168 , 0.7468321], dtype=float32)

`learn.get_preds` defaults to activation associated with the loss function, so applying softmax to the logits

In [25]:
# get the mapping from class labels to index of predicted probabilities
learn.dls.vocab

['negative', 'positive']

In [27]:
pred_proba_df = pd.DataFrame(pred_proba, columns=["negative", "positive"])
df_pseudolabels = pd.concat([imdb_df, pred_proba_df], axis=1)
df_pseudolabels.head(5)

,label,text,is_valid,negative,positive
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False,0.911338,0.088662
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False,0.253168,0.746832
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False,0.953188,0.046812
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False,0.664023,0.335977
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False,0.838614,0.161386


## Train on pseudolabels

In [33]:
blocks = (HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), RegressionBlock) # use regression block for regression tasks
dblock = DataBlock(blocks=blocks,  get_x=ColReader('text'), get_y=ColReader(['negative', 'positive']), splitter=ColSplitter()) # multiple columns for probabilities

dls = dblock.dataloaders(df_pseudolabels, bs=4)

In [34]:
dls.show_batch(dataloaders=dls, max_n=2)

,text,text_
0,"Raising Victor Vargas: A Review<br /><br />You know, Raising Victor Vargas is like sticking your hands into a big, steaming bowl of oatmeal. It's warm and gooey, but you're not sure if it feels right. Try as I might, no matter how warm and gooey Raising Victor Vargas became I was always aware that something didn't quite feel right. Victor Vargas suffers from a certain overconfidence on the director's part. Apparently, the director thought that the ethnic backdrop of a Latino family on the lower east side, and an idyllic storyline would make the film critic proof. He was right, but it didn't fool me. Raising Victor Vargas is the story about a seventeen-year old boy called, you guessed it, Victor Vargas (Victor Rasuk) who lives his teenage years chasing more skirt than the Rolling Stones could do in all the years they've toured. The movie starts off in `Ugly Fat' Donna's bedroom where Victor is sure to seduce her, but a cry from outside disrupts his plans when his best-friend Harold (Kevin Rivera) comes-a-looking for him. Caught in the attempt by Harold and his sister, Victor Vargas runs off for damage control. Yet even with the embarrassing implication that he's been boffing the homeliest girl in the neighborhood, nothing dissuades young Victor from going off on the hunt for more fresh meat. On a hot, New York City day they make way to the local public swimming pool where Victor's eyes catch a glimpse of the lovely young nymph Judy (Judy Marte), who's not just pretty, but a strong and independent too. The relationship that develops between Victor and Judy becomes the focus of the film. The story also focuses on Victor's family that is comprised of his grandmother or abuelita (Altagracia Guzman), his brother Nino (also played by real life brother to Victor, Silvestre Rasuk) and his sister Vicky (Krystal Rodriguez). The action follows Victor between scenes with Judy and scenes with his family. Victor tries to cope with being an oversexed pimp-daddy, his feelings for Judy and his grandmother's conservative Catholic upbringing.<br /><br />The problems that arise from Raising Victor Vargas are a few, but glaring errors. Throughout the film you get to know certain characters like Vicky, Nino, Grandma,","(0.640172004699707, 0.3598279654979706)"
1,"THE SHOP AROUND THE CORNER is one of the sweetest and most feel-good romantic comedies ever made. There's just no getting around that, and it's hard to actually put one's feeling for this film into words. It's not one of those films that tries too hard, nor does it come up with the oddest possible scenarios to get the two protagonists together in the end. In fact, all its charm is innate, contained within the characters and the setting and the plot... which is highly believable to boot. It's easy to think that such a love story, as beautiful as any other ever told, *could* happen to you... a feeling you don't often get from other romantic comedies, however sweet and heart-warming they may be. <br /><br />Alfred Kralik (James Stewart) and Clara Novak (Margaret Sullavan) don't have the most auspicious of first meetings when she arrives in the shop (Matuschek & Co.) he's been working in for the past nine years, asking for a job. They clash from the very beginning, mostly over a cigarette box that plays music when it's opened--he thinks it's a ludicrous idea; she makes one big sell of it and gets hired. Their bickering takes them through the next six months, even as they both (unconsciously, of course!) fall in love with each other when they share their souls and minds in letters passed through PO Box 237. This would be a pretty thin plotline to base an entire film on, except that THE SHOP AROUND THE CORNER is expertly fleshed-out with a brilliant supporting cast made up of entirely engaging characters, from the fatherly but lonely Hugo Matuschek (Frank Morgan) himself, who learns that his shop really is his home; Pirovitch (Felix Bressart), Kralik's sidekick and friend who always

In [36]:
# accuracy for pseudolabels
def pseudolabel_accuracy(pred, target):
    return (pred.argmax(dim=1) == target.argmax(dim=1)).float().mean()


In [39]:
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                loss_func=MSELossFlat(),
                metrics=[pseudolabel_accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter).to_fp16()

learn.unfreeze()

cbs = [
    GradientAccumulation(n_acc=32),
    ReduceLROnPlateau(monitor='pseudolabel_accuracy', factor=5, patience=5),
    EarlyStoppingCallback(monitor='pseudolabel_accuracy', min_delta=0.01, patience=7),
    SaveModelCallback(monitor="pseudolabel_accuracy", fname="model_1")
]



In [40]:
learn.fit_one_cycle(
    3,
    lr_max=3e-5,
    cbs = cbs
)


epoch,train_loss,valid_loss,pseudolabel_accuracy,time
0,0.071761,0.017610,0.970000,00:15
1,0.020060,0.006479,0.990000,00:15
2,0.015241,0.006530,0.995000,00:15


Better model found at epoch 0 with pseudolabel_accuracy value: 0.9700000286102295.
Better model found at epoch 1 with pseudolabel_accuracy value: 0.9900000095367432.
Better model found at epoch 2 with pseudolabel_accuracy value: 0.9950000047683716.


check results

In [45]:
df_pseudolabels.head(5)

,label,text,is_valid,negative,positive
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False,0.911338,0.088662
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False,0.253168,0.746832
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False,0.953188,0.046812
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False,0.664023,0.335977
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False,0.838614,0.161386


In [41]:
test_dl = learn.dls.test_dl(list(imdb_df["text"]))
pred_proba, _ = learn.get_preds(dl=test_dl, with_decoded=False)

In [42]:
pred_proba = pred_proba.cpu().numpy()
pred_proba.shape

(1000, 2)

test accuracy with actual labels

In [54]:
preds = pred_proba.argmax(axis=1)
pred_labels = ["negative" if p == 0 else "positive" for p in preds ]
print(len(pred_labels))
pred_labels[:5]


1000


['negative', 'positive', 'negative', 'negative', 'negative']

In [60]:
df_distilled_preds = pd.DataFrame(data=pred_labels, columns=["self_distilled_pred_label"])
df_performance = pd.concat([imdb_df, df_distilled_preds], axis=1)
df_performance_val = df_performance[df_performance["is_valid"]==True]
df_performance_val.head(5)

,label,text,is_valid,self_distilled_pred_label
800,positive,"This very funny British comedy shows what might happen if a section of London, in this case Pimlico, were to declare itself independent from the rest of the UK and its laws, taxes & post-war restrictions. Merry mayhem is what would happen.<br /><br />The explosion of a wartime bomb leads to the discovery of ancient documents which show that Pimlico was ceded to the Duchy of Burgundy centuries ago, a small historical footnote long since forgotten. To the new Burgundians, however, this is an unexpected opportunity to live as they please, free from any interference from Whitehall.<br /><br />...",True,positive
801,positive,"I saw this movie once as a kid on the late-late show and fell in love with it.<br /><br />It took 30+ years, but I recently did find it on DVD - it wasn't cheap, either - in a catalog that specialized in war movies. We watched it last night for the first time. The audio was good, however it was grainy and had the trailers between reels. Even so, it was better than I remembered it. I was also impressed at how true it was to the play.<br /><br />The catalog is around here someplace. If you're sincere in finding it, fire me a missive and I'll see if I can get you the info. cartwrightbride@yah...",True,positive
802,positive,"This is, in my opinion, a very good film, especially for Michael Jackson lovers. It contains a message on drugs, stunning special effects, and an awesome music video.<br /><br />The main film is centered around the song and music video 'Smooth Criminal.' Unlike the four-minute music video, it is normal speed and, in my opinion, much easier to watch.<br /><br />The plot is rather weird, however. Michael Jackson plays a magical 'gangster' that, when he sees a shooting star, he transforms into a piece of machinery. Throughout the film, he transforms into a race car, a giant robot, and a space...",True,positive
803,positive,"In Iran, women are not permitted to attend men's sporting events, apparently to ""protect"" them from all the cursing and foul language they might hear emanating from the male fans (so since men can't restrain or behave themselves, women are forced to suffer. Go figure.). ""Offside"" tells the tale of a half dozen or so young women who, dressed like men, attempt to sneak into the high-stakes match between Iran and Bahrain that, in 2005, qualified Iran to go to the World Cup (the movie was actually filmed in large part during that game).<br /><br />""Offside"" is a slice-of-life comedy that will ...",True,positive
804,positive,"""In April 1946, the University of Chicago agreed to operate Argonne National Laboratory, with an association of Midwestern universities offering to sponsor the research. Argonne thereby became the first ""national"" laboratory. It did not, however, remain at its original location in the Argonne forest. In 1947, it moved farther west from the ""Windy City"" to a new site on Illinois farmland. When Alvin Weinberg visited Argonne's director, Walter Zinn, in 1947, he asked him what kind of reactor was to be built at the new site. When Zinn described a heavy-water reactor operating at one-tenth the...",True,negative


In [66]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [64]:
accuracy_score(df_performance_val["label"], df_performance_val["self_distilled_pred_label"])

0.905

In [68]:
confusion_matrix(df_performance_val["label"], df_performance_val["self_distilled_pred_label"])

array([[104,   3],
       [ 16,  77]])